## Replacing B factor with normalized scores

This script will take an input PDB and replace its b-factor column with values from a table. This is useful for visualization in PyMOL.

In [1]:
# Load libraries
import re
import os
from collections import OrderedDict
import math
from Bio.PDB import *
from Bio import SeqIO
from Bio.Seq import *
from Bio.SeqRecord import *
import csv
import numpy as np
import pandas as pd

In [2]:
# Define helper functions
def parse_pdb_line(pdb_line):
    '''This function will receive a line from a PDB file and parse it as a list. It will do so based on the
    PDB format explanation from this site:

    https://www.cgl.ucsf.edu/chimera/docs/UsersGuide/tutorials/pdbintro.html.
    '''
    atom = pdb_line[0:4].strip(' ')
    atom_num = pdb_line[6:11].strip(' ')
    atom_name = pdb_line[12:16].strip(' ')
    resname = pdb_line[17:20].strip(' ')
    chain = pdb_line[21]
    res_num = pdb_line[22:26].strip(' ')
    x = pdb_line[30:38].strip(' ')
    y = pdb_line[38:46].strip(' ')
    z = pdb_line[46:54].strip(' ')

    return [atom, atom_num, atom_name, resname, chain, res_num, x, y, z]

##################################

def replace_b_factor(pdb_infile, in_dict, outfile):
    '''This function uses an input PDB file and replaces b-factors with values from
    a dictionary.'''
    in_handle = open(pdb_infile, 'r')
    out_handle = open(outfile, 'w')

    for line in in_handle:
        if line.startswith('ATOM'):

            # Parse the line
            parsed_line = parse_pdb_line(line)

            chain = parsed_line[4]
            resid = int(parsed_line[5])

            # Replace the b-factor
            dict_sasa = str(in_dict[chain][resid]) +'0'
            final_sasa = (6-len(dict_sasa))*' ' + dict_sasa

            # final_line = line.replace(line[60:66], final_sasa)
            final_line = line[0:60] + final_sasa + line[66:]
            out_handle.write(final_line)

    # Close the outfile
    out_handle.close()

##################################

def replace_admin(pdb_file, outfile, score_file):
    '''This function receives the files and feeds the information to the replace_b_factor function.'''
    ## Read the scores
    diffNormScores = pd.read_csv(score_file, sep = '\t', index_col = None)
    
    print('Maximum:', max(diffNormScores[diffNormScores.columns[1]]))
    print('Minimum:', min(diffNormScores[diffNormScores.columns[1]]))
    print('-------------')
    
    ## Convert the pandas dataframe to a dictionary compatible with the previous function
    diffNormScores_dict = {'A':{}, 
                          'B':{},
                          'C':{},
                          'D':{}}

    # Assign a zero to position one (no data for it)
    diffNormScores_dict['A'][1] = round(0, 2)
    diffNormScores_dict['B'][1] = round(0, 2)
    diffNormScores_dict['C'][1] = round(0, 2)
    diffNormScores_dict['D'][1] = round(0, 2)

    for index, row in diffNormScores.iterrows():

        ### For the new file
        ## Add values to dictionary
        diffNormScores_dict['A'][int(row['Position'])] = round(row[1], 2)
        diffNormScores_dict['B'][int(row['Position'])] = round(row[1], 2)
        diffNormScores_dict['C'][int(row['Position'])] = round(row[1], 2)
        diffNormScores_dict['D'][int(row['Position'])] = round(row[1], 2)

    # Replace the b-factors and write the output file
    replace_b_factor(pdb_file, diffNormScores_dict, outfile)
    

## Prepare a visualization of the mean of differences in fitness effects

In [3]:
### For the old file with the differences of medians
## Define input variables
# pdb_file = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Data/Structures/2rk1_bio.pdb'
# outfile = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Analysis/log2_FC/Residues_diffNormScores_avg3reps/2rk1_medDiffNormScores_ara0.4_ara0.01.pdb'
# diffNormScores = pd.read_csv('/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Analysis/log2_FC/Residues_diffNormScores_avg3reps/medDiffNormScores_avg3reps_ara0.4_ara0.01.txt', 
#                              sep = '\t', index_col = None)

### For the new file with the mean of differences
## Define input variables
# pdb_file = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Data/Structures/2rk1_bio.pdb'
# pdb_file = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/PDB_files/2rk1_bio.pdb'
# pdb_file = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/PDB_files/2rk1_bio.pdb'
pdb_file = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/PDB_files/DfrB1_alphafold_rank1_goodCoords.pdb'


# outfile = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Analysis/log2_FC/Residues_diffNormScores_avg3reps/2rk1_meanDiffFitEff_ara0.4_ara0.01.pdb'
# outfile = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Analysis/log2_FC/Residues_diffNormScores_avg3reps/2rk1_meanDiffFitEff_ara0.2_ara0.01_new.pdb'
outfile = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/PDB_files/2rk1_meanDiffFitEff_ara0.2_ara0.01_new_af2.pdb'

# diffNormScores = pd.read_csv('/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Analysis/log2_FC/Residues_diffNormScores_avg3reps/meanDiffFitEff_ara0.4_ara0.01.txt', sep = '\t', index_col = None)
# diffNormScores = pd.read_csv('/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Analysis/log2_FC/Residues_diffNormScores_avg3reps/meanDiffFitEff_ara0.2_ara0.01.txt', 
# diffNormScores = pd.read_csv('/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Analysis/log2_FC/Residues_diffNormScores_avg3reps/meanDiffFitEff_ara0.2_ara0.01_new.txt', 
diffNormScores = pd.read_csv('/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/Tables/meanDiffFitEff_ara0.2_ara0.01_new.txt', 
                             sep = '\t', index_col = None)

In [4]:
diffNormScores

,Position,Mean_of_differences_fit_eff
0,2,0.260500
1,3,0.141325
2,4,-0.035905
3,5,-0.008297
4,6,-0.016305
...,...,...
72,74,-0.111244
73,75,-0.366496
74,76,-0.274595
75,77,-0.272907


In [5]:
print(max(diffNormScores['Mean_of_differences_fit_eff']))
print(min(diffNormScores['Mean_of_differences_fit_eff']))

0.260499692636749
-0.366495939615364


In [6]:
diffNormScores.describe()

,Position,Mean_of_differences_fit_eff
count,77.000000,77.000000
mean,40.000000,-0.105811
std,22.371857,0.115096
min,2.000000,-0.366496
25%,21.000000,-0.173476
50%,40.000000,-0.087454
75%,59.000000,-0.027798
max,78.000000,0.260500


In [7]:
## Convert the pandas dataframe to a dictionary compatible with the previous function
diffNormScores_dict = {'A':{}, 
                      'B':{},
                      'C':{},
                      'D':{}}

# Assign a zero to position one (no data for it)
diffNormScores_dict['A'][1] = round(0, 2)
diffNormScores_dict['B'][1] = round(0, 2)
diffNormScores_dict['C'][1] = round(0, 2)
diffNormScores_dict['D'][1] = round(0, 2)

for index, row in diffNormScores.iterrows():
    
    ### For the old file
    ## Add values to dictionary
    # diffNormScores_dict['A'][int(row['position'])] = round(row['med_diff_fit_eff'], 2)
    # diffNormScores_dict['B'][int(row['position'])] = round(row['med_diff_fit_eff'], 2)
    # diffNormScores_dict['C'][int(row['position'])] = round(row['med_diff_fit_eff'], 2)
    # diffNormScores_dict['D'][int(row['position'])] = round(row['med_diff_fit_eff'], 2)
    
    ### For the new file
    ## Add values to dictionary
    diffNormScores_dict['A'][int(row['Position'])] = round(row['Mean_of_differences_fit_eff'], 2)
    diffNormScores_dict['B'][int(row['Position'])] = round(row['Mean_of_differences_fit_eff'], 2)
    diffNormScores_dict['C'][int(row['Position'])] = round(row['Mean_of_differences_fit_eff'], 2)
    diffNormScores_dict['D'][int(row['Position'])] = round(row['Mean_of_differences_fit_eff'], 2)


In [8]:
diffNormScores_dict

{'A': {1: 0,
  2: 0.26,
  3: 0.14,
  4: -0.04,
  5: -0.01,
  6: -0.02,
  7: -0.03,
  8: 0.01,
  9: 0.01,
  10: -0.01,
  11: 0.0,
  12: 0.02,
  13: 0.04,
  14: 0.02,
  15: 0.0,
  16: -0.1,
  17: -0.01,
  18: -0.24,
  19: -0.34,
  20: 0.04,
  21: -0.01,
  22: -0.16,
  23: -0.13,
  24: -0.13,
  25: -0.03,
  26: -0.19,
  27: -0.28,
  28: -0.26,
  29: -0.3,
  30: -0.07,
  31: -0.21,
  32: -0.04,
  33: -0.19,
  34: -0.19,
  35: -0.05,
  36: -0.07,
  37: -0.21,
  38: -0.01,
  39: -0.25,
  40: -0.06,
  41: -0.05,
  42: -0.07,
  43: -0.1,
  44: -0.09,
  45: -0.18,
  46: -0.04,
  47: -0.15,
  48: -0.08,
  49: -0.03,
  50: -0.29,
  51: -0.13,
  52: -0.34,
  53: -0.15,
  54: -0.07,
  55: -0.05,
  56: -0.09,
  57: -0.1,
  58: -0.09,
  59: -0.09,
  60: -0.36,
  61: -0.14,
  62: -0.09,
  63: -0.17,
  64: -0.11,
  65: -0.11,
  66: -0.08,
  67: -0.01,
  68: -0.06,
  69: -0.07,
  70: -0.03,
  71: -0.26,
  72: -0.17,
  73: -0.17,
  74: -0.11,
  75: -0.37,
  76: -0.27,
  77: -0.27,
  78: -0.02},
 'B': {1:

In [9]:
# Replace the b-factors and write the output file
replace_b_factor(pdb_file, diffNormScores_dict, outfile)

## Repeat the above to get some similar figures for the mean fitness effects for ara 0.01

In [4]:
# Define input variables
pdb_file = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/PDB_files/DfrB1_alphafold_rank1_goodCoords.pdb'

outfile = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/PDB_files/2rk1_meanFitEff_ara0.01_new_af2.pdb'

meanNormScores = pd.read_csv('/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/Tables/meanFitEff_ara0.01_new.txt', 
                             sep = '\t', index_col = None)

In [5]:
meanNormScores

,Position,mean_s
0,2,0.316419
1,3,0.145916
2,4,-0.010840
3,5,0.030180
4,6,0.017466
...,...,...
72,74,-0.637922
73,75,-0.512030
74,76,-0.300449
75,77,-0.319971


In [6]:
print(max(meanNormScores['mean_s']))
print(min(meanNormScores['mean_s']))

0.316419346168411
-0.703212781509524


In [7]:
## Convert the pandas dataframe to a dictionary compatible with the previous function
meanNormScores_dict = {'A':{}, 
                      'B':{},
                      'C':{},
                      'D':{}}

# Assign a zero to position one
meanNormScores_dict['A'][1] = round(0, 2)
meanNormScores_dict['B'][1] = round(0, 2)
meanNormScores_dict['C'][1] = round(0, 2)
meanNormScores_dict['D'][1] = round(0, 2)

for index, row in meanNormScores.iterrows():
    
    # Add values to dictionary
    meanNormScores_dict['A'][int(row['Position'])] = round(row['mean_s'], 2)
    meanNormScores_dict['B'][int(row['Position'])] = round(row['mean_s'], 2)
    meanNormScores_dict['C'][int(row['Position'])] = round(row['mean_s'], 2)
    meanNormScores_dict['D'][int(row['Position'])] = round(row['mean_s'], 2)

In [8]:
meanNormScores_dict

{'A': {1: 0,
  2: 0.32,
  3: 0.15,
  4: -0.01,
  5: 0.03,
  6: 0.02,
  7: 0.01,
  8: 0.05,
  9: 0.05,
  10: 0.02,
  11: 0.04,
  12: 0.06,
  13: 0.08,
  14: 0.06,
  15: 0.04,
  16: -0.08,
  17: 0.02,
  18: -0.27,
  19: -0.35,
  20: 0.08,
  21: 0.03,
  22: -0.17,
  23: -0.16,
  24: -0.6,
  25: -0.02,
  26: -0.28,
  27: -0.51,
  28: -0.42,
  29: -0.42,
  30: -0.67,
  31: -0.55,
  32: -0.66,
  33: -0.48,
  34: -0.28,
  35: -0.7,
  36: -0.61,
  37: -0.26,
  38: -0.67,
  39: -0.33,
  40: -0.68,
  41: -0.04,
  42: -0.61,
  43: -0.65,
  44: -0.65,
  45: -0.5,
  46: -0.69,
  47: -0.24,
  48: -0.66,
  49: 0.0,
  50: -0.45,
  51: -0.67,
  52: -0.39,
  53: -0.59,
  54: -0.69,
  55: -0.64,
  56: -0.66,
  57: -0.66,
  58: -0.67,
  59: -0.63,
  60: -0.48,
  61: -0.18,
  62: -0.69,
  63: -0.16,
  64: -0.66,
  65: -0.64,
  66: -0.7,
  67: -0.61,
  68: -0.68,
  69: -0.66,
  70: -0.65,
  71: -0.4,
  72: -0.2,
  73: -0.61,
  74: -0.64,
  75: -0.51,
  76: -0.3,
  77: -0.32,
  78: 0.03},
 'B': {1: 0,
  2: 0

In [9]:
# Replace the b-factors and write the output file
replace_b_factor(pdb_file, meanNormScores_dict, outfile)

## Repeat the above to get some similar figures for the mean fitness effects for ara 0.025

In [10]:
# Define input variables
pdb_file = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/PDB_files/DfrB1_alphafold_rank1_goodCoords.pdb'

outfile = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/PDB_files/2rk1_meanFitEff_ara0.025_new_af2.pdb'

meanNormScores = pd.read_csv('/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/Tables/meanFitEff_ara0.025_new.txt', 
                             sep = '\t', index_col = None)

In [11]:
meanNormScores

,Position,mean_s
0,2,0.207896
1,3,0.070779
2,4,0.013145
3,5,0.041664
4,6,0.031281
...,...,...
72,74,-0.586290
73,75,-0.447970
74,76,-0.198343
75,77,-0.240331


In [12]:
print(max(meanNormScores['mean_s']))
print(min(meanNormScores['mean_s']))

0.207895683120582
-0.6699673069047621


In [13]:
## Convert the pandas dataframe to a dictionary compatible with the previous function
meanNormScores_dict = {'A':{}, 
                      'B':{},
                      'C':{},
                      'D':{}}

# Assign a zero to position one
meanNormScores_dict['A'][1] = round(0, 2)
meanNormScores_dict['B'][1] = round(0, 2)
meanNormScores_dict['C'][1] = round(0, 2)
meanNormScores_dict['D'][1] = round(0, 2)

for index, row in meanNormScores.iterrows():
    
    # Add values to dictionary
    meanNormScores_dict['A'][int(row['Position'])] = round(row['mean_s'], 2)
    meanNormScores_dict['B'][int(row['Position'])] = round(row['mean_s'], 2)
    meanNormScores_dict['C'][int(row['Position'])] = round(row['mean_s'], 2)
    meanNormScores_dict['D'][int(row['Position'])] = round(row['mean_s'], 2)

In [14]:
meanNormScores_dict

{'A': {1: 0,
  2: 0.21,
  3: 0.07,
  4: 0.01,
  5: 0.04,
  6: 0.03,
  7: 0.04,
  8: 0.05,
  9: 0.05,
  10: 0.03,
  11: 0.04,
  12: 0.06,
  13: 0.07,
  14: 0.06,
  15: 0.04,
  16: -0.02,
  17: 0.04,
  18: -0.2,
  19: -0.25,
  20: 0.07,
  21: 0.04,
  22: -0.1,
  23: -0.11,
  24: -0.56,
  25: -0.0,
  26: -0.23,
  27: -0.45,
  28: -0.36,
  29: -0.34,
  30: -0.61,
  31: -0.53,
  32: -0.62,
  33: -0.45,
  34: -0.21,
  35: -0.66,
  36: -0.59,
  37: -0.2,
  38: -0.66,
  39: -0.26,
  40: -0.64,
  41: -0.02,
  42: -0.57,
  43: -0.6,
  44: -0.61,
  45: -0.47,
  46: -0.67,
  47: -0.2,
  48: -0.63,
  49: 0.02,
  50: -0.38,
  51: -0.63,
  52: -0.3,
  53: -0.59,
  54: -0.65,
  55: -0.62,
  56: -0.61,
  57: -0.62,
  58: -0.63,
  59: -0.58,
  60: -0.42,
  61: -0.13,
  62: -0.65,
  63: -0.09,
  64: -0.61,
  65: -0.6,
  66: -0.66,
  67: -0.6,
  68: -0.64,
  69: -0.63,
  70: -0.64,
  71: -0.32,
  72: -0.12,
  73: -0.59,
  74: -0.59,
  75: -0.45,
  76: -0.2,
  77: -0.24,
  78: 0.05},
 'B': {1: 0,
  2: 0.21

In [15]:
# Replace the b-factors and write the output file
replace_b_factor(pdb_file, meanNormScores_dict, outfile)

## Repeat the above to get some similar figures for the mean fitness effects for ara 0.05

In [16]:
# Define input variables
pdb_file = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/PDB_files/DfrB1_alphafold_rank1_goodCoords.pdb'

outfile = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/PDB_files/2rk1_meanFitEff_ara0.05_new_af2.pdb'

meanNormScores = pd.read_csv('/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/Tables/meanFitEff_ara0.05_new.txt', 
                             sep = '\t', index_col = None)

In [17]:
meanNormScores

,Position,mean_s
0,2,0.162427
1,3,0.067558
2,4,0.046698
3,5,0.067805
4,6,0.058579
...,...,...
72,74,-0.534740
73,75,-0.283077
74,76,-0.039016
75,77,-0.085916


In [18]:
print(max(meanNormScores['mean_s']))
print(min(meanNormScores['mean_s']))

0.162427442597047
-0.630638255933333


In [19]:
## Convert the pandas dataframe to a dictionary compatible with the previous function
meanNormScores_dict = {'A':{}, 
                      'B':{},
                      'C':{},
                      'D':{}}

# Assign a zero to position one
meanNormScores_dict['A'][1] = round(0, 2)
meanNormScores_dict['B'][1] = round(0, 2)
meanNormScores_dict['C'][1] = round(0, 2)
meanNormScores_dict['D'][1] = round(0, 2)

for index, row in meanNormScores.iterrows():
    
    # Add values to dictionary
    meanNormScores_dict['A'][int(row['Position'])] = round(row['mean_s'], 2)
    meanNormScores_dict['B'][int(row['Position'])] = round(row['mean_s'], 2)
    meanNormScores_dict['C'][int(row['Position'])] = round(row['mean_s'], 2)
    meanNormScores_dict['D'][int(row['Position'])] = round(row['mean_s'], 2)

In [20]:
meanNormScores_dict

{'A': {1: 0,
  2: 0.16,
  3: 0.07,
  4: 0.05,
  5: 0.07,
  6: 0.06,
  7: 0.07,
  8: 0.07,
  9: 0.07,
  10: 0.06,
  11: 0.07,
  12: 0.07,
  13: 0.08,
  14: 0.07,
  15: 0.07,
  16: 0.04,
  17: 0.06,
  18: -0.05,
  19: -0.05,
  20: 0.09,
  21: 0.07,
  22: -0.01,
  23: -0.03,
  24: -0.47,
  25: 0.03,
  26: -0.12,
  27: -0.3,
  28: -0.21,
  29: -0.17,
  30: -0.56,
  31: -0.43,
  32: -0.57,
  33: -0.34,
  34: -0.11,
  35: -0.63,
  36: -0.54,
  37: -0.09,
  38: -0.61,
  39: -0.16,
  40: -0.61,
  41: 0.02,
  42: -0.5,
  43: -0.55,
  44: -0.57,
  45: -0.39,
  46: -0.61,
  47: -0.12,
  48: -0.57,
  49: 0.06,
  50: -0.22,
  51: -0.56,
  52: -0.13,
  53: -0.52,
  54: -0.58,
  55: -0.56,
  56: -0.57,
  57: -0.56,
  58: -0.58,
  59: -0.53,
  60: -0.24,
  61: -0.05,
  62: -0.61,
  63: -0.01,
  64: -0.57,
  65: -0.55,
  66: -0.61,
  67: -0.59,
  68: -0.61,
  69: -0.57,
  70: -0.61,
  71: -0.19,
  72: -0.04,
  73: -0.53,
  74: -0.53,
  75: -0.28,
  76: -0.04,
  77: -0.09,
  78: 0.08},
 'B': {1: 0,
  2:

In [21]:
# Replace the b-factors and write the output file
replace_b_factor(pdb_file, meanNormScores_dict, outfile)

## Repeat the above to get some similar figures for the mean fitness effects for ara 0.2

In [22]:
# Define input variables
pdb_file = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/PDB_files/DfrB1_alphafold_rank1_goodCoords.pdb'

outfile = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/PDB_files/2rk1_meanFitEff_ara0.2_new_af2.pdb'

meanNormScores = pd.read_csv('/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/Tables/meanFitEff_ara0.2_new.txt', 
                             sep = '\t', index_col = None)

In [23]:
meanNormScores

,Position,mean_s
0,2,0.060120
1,3,0.006184
2,4,0.018729
3,5,0.031891
4,6,0.027564
...,...,...
72,74,-0.527879
73,75,-0.133923
74,76,-0.030758
75,77,-0.046062


In [24]:
print(max(meanNormScores['mean_s']))
print(min(meanNormScores['mean_s']))

0.060120033761129996
-0.666477511238095


In [25]:
## Convert the pandas dataframe to a dictionary compatible with the previous function
meanNormScores_dict = {'A':{}, 
                      'B':{},
                      'C':{},
                      'D':{}}

meanNormScores_dict['A'][1] = round(0, 2)
meanNormScores_dict['B'][1] = round(0, 2)
meanNormScores_dict['C'][1] = round(0, 2)
meanNormScores_dict['D'][1] = round(0, 2)

for index, row in meanNormScores.iterrows():
    
    # Add values to dictionary
    meanNormScores_dict['A'][int(row['Position'])] = round(row['mean_s'], 2)
    meanNormScores_dict['B'][int(row['Position'])] = round(row['mean_s'], 2)
    meanNormScores_dict['C'][int(row['Position'])] = round(row['mean_s'], 2)
    meanNormScores_dict['D'][int(row['Position'])] = round(row['mean_s'], 2)

In [26]:
meanNormScores_dict

{'A': {1: 0,
  2: 0.06,
  3: 0.01,
  4: 0.02,
  5: 0.03,
  6: 0.03,
  7: 0.04,
  8: 0.03,
  9: 0.03,
  10: 0.03,
  11: 0.03,
  12: 0.03,
  13: 0.04,
  14: 0.03,
  15: 0.03,
  16: 0.02,
  17: 0.03,
  18: -0.03,
  19: -0.01,
  20: 0.04,
  21: 0.03,
  22: -0.02,
  23: -0.03,
  24: -0.46,
  25: 0.0,
  26: -0.09,
  27: -0.22,
  28: -0.16,
  29: -0.12,
  30: -0.6,
  31: -0.33,
  32: -0.63,
  33: -0.28,
  34: -0.09,
  35: -0.65,
  36: -0.54,
  37: -0.06,
  38: -0.67,
  39: -0.08,
  40: -0.63,
  41: 0.0,
  42: -0.54,
  43: -0.54,
  44: -0.58,
  45: -0.31,
  46: -0.66,
  47: -0.1,
  48: -0.58,
  49: 0.03,
  50: -0.16,
  51: -0.55,
  52: -0.05,
  53: -0.43,
  54: -0.62,
  55: -0.59,
  56: -0.57,
  57: -0.57,
  58: -0.58,
  59: -0.55,
  60: -0.11,
  61: -0.04,
  62: -0.6,
  63: -0.01,
  64: -0.54,
  65: -0.52,
  66: -0.62,
  67: -0.6,
  68: -0.62,
  69: -0.6,
  70: -0.63,
  71: -0.14,
  72: -0.03,
  73: -0.43,
  74: -0.53,
  75: -0.13,
  76: -0.03,
  77: -0.05,
  78: 0.05},
 'B': {1: 0,
  2: 0.06

In [27]:
# Replace the b-factors and write the output file
replace_b_factor(pdb_file, meanNormScores_dict, outfile)

## Repeat with the standard deviation of delta(s) values for each position

In [35]:
# Define input variables
pdb_file = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/PDB_files/DfrB1_alphafold_rank1_goodCoords.pdb'

outfile = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/PDB_files/2rk1_sdDiffFitEff_ara0.01_new_af2.pdb'

diffNormScores = pd.read_csv('/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/Tables/sdDiffFitEff_ara0.2_ara0.01_new.txt', 
                             sep = '\t', index_col = None)

In [36]:
diffNormScores

,Position,Sd_of_differences_fit_eff
0,2,0.096024
1,3,0.142939
2,4,0.055314
3,5,0.054494
4,6,0.063464
...,...,...
72,74,0.083375
73,75,0.201456
74,76,0.166073
75,77,0.185962


In [37]:
print(max(diffNormScores['Sd_of_differences_fit_eff']))
print(min(diffNormScores['Sd_of_differences_fit_eff']))

0.21940561408801998
0.0373380964338499


In [38]:
diffNormScores.describe()

,Position,Sd_of_differences_fit_eff
count,77.000000,77.000000
mean,40.000000,0.107626
std,22.371857,0.053601
min,2.000000,0.037338
25%,21.000000,0.058959
50%,40.000000,0.098493
75%,59.000000,0.159384
max,78.000000,0.219406


In [39]:
## Convert the pandas dataframe to a dictionary compatible with the previous function
diffNormScores_dict = {'A':{}, 
                      'B':{},
                      'C':{},
                      'D':{}}

## Assign a zero to position one
diffNormScores_dict['A'][1] = round(0, 2)
diffNormScores_dict['B'][1] = round(0, 2)
diffNormScores_dict['C'][1] = round(0, 2)
diffNormScores_dict['D'][1] = round(0, 2)

for index, row in diffNormScores.iterrows():
    
    ### For the old file
    ## Add values to dictionary
    # diffNormScores_dict['A'][int(row['position'])] = round(row['med_diff_fit_eff'], 2)
    # diffNormScores_dict['B'][int(row['position'])] = round(row['med_diff_fit_eff'], 2)
    # diffNormScores_dict['C'][int(row['position'])] = round(row['med_diff_fit_eff'], 2)
    # diffNormScores_dict['D'][int(row['position'])] = round(row['med_diff_fit_eff'], 2)
    
    ### For the new file
    ## Add values to dictionary
    diffNormScores_dict['A'][int(row['Position'])] = round(row['Sd_of_differences_fit_eff'], 2)
    diffNormScores_dict['B'][int(row['Position'])] = round(row['Sd_of_differences_fit_eff'], 2)
    diffNormScores_dict['C'][int(row['Position'])] = round(row['Sd_of_differences_fit_eff'], 2)
    diffNormScores_dict['D'][int(row['Position'])] = round(row['Sd_of_differences_fit_eff'], 2)


In [40]:
diffNormScores_dict

{'A': {1: 0,
  2: 0.1,
  3: 0.14,
  4: 0.06,
  5: 0.05,
  6: 0.06,
  7: 0.1,
  8: 0.06,
  9: 0.07,
  10: 0.05,
  11: 0.04,
  12: 0.04,
  13: 0.04,
  14: 0.05,
  15: 0.06,
  16: 0.13,
  17: 0.04,
  18: 0.2,
  19: 0.19,
  20: 0.06,
  21: 0.06,
  22: 0.14,
  23: 0.12,
  24: 0.13,
  25: 0.07,
  26: 0.15,
  27: 0.17,
  28: 0.17,
  29: 0.13,
  30: 0.1,
  31: 0.18,
  32: 0.05,
  33: 0.19,
  34: 0.16,
  35: 0.06,
  36: 0.13,
  37: 0.17,
  38: 0.06,
  39: 0.22,
  40: 0.05,
  41: 0.13,
  42: 0.05,
  43: 0.1,
  44: 0.08,
  45: 0.18,
  46: 0.04,
  47: 0.16,
  48: 0.07,
  49: 0.1,
  50: 0.17,
  51: 0.1,
  52: 0.2,
  53: 0.17,
  54: 0.06,
  55: 0.07,
  56: 0.05,
  57: 0.07,
  58: 0.08,
  59: 0.06,
  60: 0.21,
  61: 0.18,
  62: 0.1,
  63: 0.11,
  64: 0.12,
  65: 0.12,
  66: 0.08,
  67: 0.04,
  68: 0.07,
  69: 0.11,
  70: 0.06,
  71: 0.17,
  72: 0.14,
  73: 0.1,
  74: 0.08,
  75: 0.2,
  76: 0.17,
  77: 0.19,
  78: 0.04},
 'B': {1: 0,
  2: 0.1,
  3: 0.14,
  4: 0.06,
  5: 0.05,
  6: 0.06,
  7: 0.1,
  8:

In [41]:
# Replace the b-factors and write the output file
replace_b_factor(pdb_file, diffNormScores_dict, outfile)

## Use the new admin function to get PDB files for the maximum and minimum expression-dependent differences at each position

In [10]:
# Quick test to make sure the function works as intended
replace_admin(pdb_file = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/PDB_files/DfrB1_alphafold_rank1_goodCoords.pdb',
             outfile = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/PDB_files/2rk1_meanDiffFitEff_ara0.2_ara0.01_new_af2_test.pdb',
             score_file = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Figures/2022-03-24_Figures_paper/chimerax/Tables/meanDiffFitEff_ara0.2_ara0.01_new.txt')


Maximum: 0.260499692636749
Minimum: -0.366495939615364
-------------


I used the command line diff to make sure the file was identical to my old one and it worked.

In [11]:
pwd

'/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Scripts'

In [18]:
## Work with the files for the maximum effects
pdb_file = '../Figures/2022-03-24_Figures_paper/chimerax/PDB_files/DfrB1_alphafold_rank1_goodCoords.pdb'

## 0.01% arabinose
replace_admin(pdb_file = pdb_file, 
             outfile = '../Figures/2022-03-24_Figures_paper/chimerax/PDB_files/max/2rk1_max_fitEff_ara0.01_af2.pdb',
             score_file = '../Figures/2022-03-24_Figures_paper/chimerax/Tables/max/maxFitEff_ara0.01_new.txt')

## 0.025% arabinose
replace_admin(pdb_file = pdb_file, 
             outfile = '../Figures/2022-03-24_Figures_paper/chimerax/PDB_files/max/2rk1_max_fitEff_ara0.025_af2.pdb',
             score_file = '../Figures/2022-03-24_Figures_paper/chimerax/Tables/max/maxFitEff_ara0.025_new.txt')

## 0.05% arabinose
replace_admin(pdb_file = pdb_file, 
             outfile = '../Figures/2022-03-24_Figures_paper/chimerax/PDB_files/max/2rk1_max_fitEff_ara0.05_af2.pdb',
             score_file = '../Figures/2022-03-24_Figures_paper/chimerax/Tables/max/maxFitEff_ara0.05_new.txt')

## 0.2% arabinose
replace_admin(pdb_file = pdb_file, 
             outfile = '../Figures/2022-03-24_Figures_paper/chimerax/PDB_files/max/2rk1_max_fitEff_ara0.2_af2.pdb',
             score_file = '../Figures/2022-03-24_Figures_paper/chimerax/Tables/max/maxFitEff_ara0.2_new.txt')

## 0.4% arabinose
replace_admin(pdb_file = pdb_file, 
             outfile = '../Figures/2022-03-24_Figures_paper/chimerax/PDB_files/max/2rk1_max_fitEff_ara0.4_af2.pdb',
             score_file = '../Figures/2022-03-24_Figures_paper/chimerax/Tables/max/maxFitEff_ara0.4_new.txt')



Maximum: 0.40541249093074605
Minimum: -0.0007145743383116701
-------------
Maximum: 0.265663055050904
Minimum: -0.00071872994954602
-------------
Maximum: 0.223569649710326
Minimum: -0.00091124791636729
-------------
Maximum: 0.107415637473871
Minimum: -0.00046329488160937003
-------------
Maximum: 0.0627700835375569
Minimum: -0.00026147220107874
-------------


In [19]:
## Work with the files for the minimum effects
pdb_file = '../Figures/2022-03-24_Figures_paper/chimerax/PDB_files/DfrB1_alphafold_rank1_goodCoords.pdb'

## 0.01% arabinose
replace_admin(pdb_file = pdb_file, 
             outfile = '../Figures/2022-03-24_Figures_paper/chimerax/PDB_files/min/2rk1_min_fitEff_ara0.01_af2.pdb',
             score_file = '../Figures/2022-03-24_Figures_paper/chimerax/Tables/min/minFitEff_ara0.01_new.txt')

## 0.025% arabinose
replace_admin(pdb_file = pdb_file, 
             outfile = '../Figures/2022-03-24_Figures_paper/chimerax/PDB_files/min/2rk1_min_fitEff_ara0.025_af2.pdb',
             score_file = '../Figures/2022-03-24_Figures_paper/chimerax/Tables/min/minFitEff_ara0.025_new.txt')

## 0.05% arabinose
replace_admin(pdb_file = pdb_file, 
             outfile = '../Figures/2022-03-24_Figures_paper/chimerax/PDB_files/min/2rk1_min_fitEff_ara0.05_af2.pdb',
             score_file = '../Figures/2022-03-24_Figures_paper/chimerax/Tables/min/minFitEff_ara0.05_new.txt')

## 0.2% arabinose
replace_admin(pdb_file = pdb_file, 
             outfile = '../Figures/2022-03-24_Figures_paper/chimerax/PDB_files/min/2rk1_min_fitEff_ara0.2_af2.pdb',
             score_file = '../Figures/2022-03-24_Figures_paper/chimerax/Tables/min/minFitEff_ara0.2_new.txt')

## 0.4% arabinose
replace_admin(pdb_file = pdb_file, 
             outfile = '../Figures/2022-03-24_Figures_paper/chimerax/PDB_files/min/2rk1_min_fitEff_ara0.4_af2.pdb',
             score_file = '../Figures/2022-03-24_Figures_paper/chimerax/Tables/min/minFitEff_ara0.4_new.txt')



Maximum: -0.0510478908318117
Minimum: -0.954332667843951
-------------
Maximum: -0.00418364301513635
Minimum: -0.9713070467836981
-------------
Maximum: -0.00067762682980065
Minimum: -0.819845619785162
-------------
Maximum: -0.0003274344452381
Minimum: -0.8741435020180699
-------------
Maximum: -0.00295665798125407
Minimum: -0.9178172586656071
-------------


## Use the admin function to save structures for maximum, minimum, and mean deltaS at each position (s_weak - s_opt)

In [4]:
## Work with the files for the minimum effects
pdb_file = '../Figures/2022-03-24_Figures_paper/chimerax/PDB_files/DfrB1_alphafold_rank1_goodCoords.pdb'

## Max delta s
replace_admin(pdb_file = pdb_file, 
             outfile = '../Figures/2022-03-24_Figures_paper/chimerax/PDB_files/deltas_weak_opt/2rk1_max_deltaS_ara0.01_ara0.2_af2.pdb',
             score_file = '../Figures/2022-03-24_Figures_paper/chimerax/Tables/deltas_weak_opt/max_deltaS_ara0.2_ara0.01.txt')

## Min delta s
replace_admin(pdb_file = pdb_file, 
             outfile = '../Figures/2022-03-24_Figures_paper/chimerax/PDB_files/deltas_weak_opt/2rk1_min_deltaS_ara0.01_ara0.2_af2.pdb',
             score_file = '../Figures/2022-03-24_Figures_paper/chimerax/Tables/deltas_weak_opt/min_deltaS_ara0.2_ara0.01.txt')

## Mean delta s
replace_admin(pdb_file = pdb_file, 
             outfile = '../Figures/2022-03-24_Figures_paper/chimerax/PDB_files/deltas_weak_opt/2rk1_mean_deltaS_ara0.01_ara0.2_af2.pdb',
             score_file = '../Figures/2022-03-24_Figures_paper/chimerax/Tables/deltas_weak_opt/mean_deltaS_ara0.2_ara0.01.txt')


Maximum: 0.333542433249573
Minimum: -0.00031380209463774
-------------
Maximum: 0.00334569929397661
Minimum: -0.6927089222989661
-------------
Maximum: 0.258138998600794
Minimum: -0.347336176416463
-------------


## Stuff I am not using anymore

## Let's repeat the above to get some similar figures for the entropy values

In [35]:
# Define input variables
pdb_file = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Data/Structures/2rk1_bio.pdb'
outfile = '/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Analysis/log2_FC/medNormScores_avg3reps/2rk1_entropy.pdb'
medNormScores = pd.read_csv('/media/axelle/afe8c733-963d-4db8-a2ee-551a0b73c9d7/Angel/PhD_projects/R67_DMS_December2020/Analysis/log2_FC/medNormScores_avg3reps/medianFitEffects_ara0.4.txt', 
                             sep = '\t', index_col = None)

In [36]:
## Convert the pandas dataframe to a dictionary compatible with the previous function
medNormScores_dict = {'A':{}, 
                      'B':{},
                      'C':{},
                      'D':{}}

for index, row in medNormScores.iterrows():
    
    # Add values to dictionary
    medNormScores_dict['A'][int(row['position'])] = round(row['Entropy'], 2)
    medNormScores_dict['B'][int(row['position'])] = round(row['Entropy'], 2)
    medNormScores_dict['C'][int(row['position'])] = round(row['Entropy'], 2)
    medNormScores_dict['D'][int(row['position'])] = round(row['Entropy'], 2)

In [37]:
medNormScores_dict

{'A': {2: 1.61,
  3: 1.28,
  4: 1.27,
  5: 1.52,
  6: 0.91,
  7: 0.64,
  8: 0.5,
  9: 0.91,
  10: 1.13,
  11: 0.98,
  12: 0.84,
  13: 0.46,
  14: 0.69,
  15: 1.51,
  16: 0.75,
  17: 1.27,
  18: 1.3,
  19: 0.37,
  20: 1.0,
  21: 1.36,
  22: 0.47,
  23: 0.84,
  24: 0.14,
  25: 1.18,
  26: 1.1,
  27: 0.0,
  28: 0.21,
  29: 0.6,
  30: 0.35,
  31: 0.48,
  32: 0.0,
  33: 0.21,
  34: 0.51,
  35: 0.31,
  36: 0.28,
  37: 0.66,
  38: 0.0,
  39: 0.63,
  40: 0.0,
  41: 0.88,
  42: 0.58,
  43: 0.0,
  44: 0.0,
  45: 0.26,
  46: 0.0,
  47: 0.69,
  48: 0.0,
  49: 1.5,
  50: 0.19,
  51: 0.0,
  52: 0.43,
  53: 0.49,
  54: 0.0,
  55: 0.0,
  56: 0.12,
  57: 0.34,
  58: 0.07,
  59: 0.07,
  60: 0.64,
  61: 1.17,
  62: 0.46,
  63: 0.26,
  64: 0.14,
  65: 0.07,
  66: 0.0,
  67: 0.0,
  68: 0.0,
  69: 0.0,
  70: 0.0,
  71: 1.03,
  72: 0.98,
  73: 0.0,
  74: 0.38,
  75: 0.26,
  76: 0.54,
  77: 1.11,
  78: 1.37},
 'B': {2: 1.61,
  3: 1.28,
  4: 1.27,
  5: 1.52,
  6: 0.91,
  7: 0.64,
  8: 0.5,
  9: 0.91,
  10: 1.1

In [38]:
# Replace the b-factors and write the output file
replace_b_factor(pdb_file, medNormScores_dict, outfile)